<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/YOLOv5_interference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5 interference**

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [80]:
%cd /gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco

/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco


In [40]:
%%writefile requirements.txt

# pip install -r requirements.txt

# Base ----------------------------------------
matplotlib>=3.2.2
numpy>=1.18.5
opencv-python-headless>=4.6.0.66
Pillow>=7.1.2
PyYAML>=5.3.1
##requests>=2.23.0
scipy>=1.4.1
# torch>=1.7.0
# torchvision>=0.8.1
tqdm>=4.41.0

# Logging -------------------------------------
##tensorboard>=2.4.1
# wandb

# Plotting ------------------------------------
##pandas>=1.1.4
##seaborn>=0.11.0

# Export --------------------------------------
# coremltools>=4.1  # CoreML export
# onnx>=1.9.0  # ONNX export
# onnx-simplifier>=0.3.6  # ONNX simplifier
# scikit-learn==0.19.2  # CoreML quantization
# tensorflow>=2.4.1  # TFLite export
# tensorflowjs>=3.9.0  # TF.js export

# Extras --------------------------------------
# albumentations>=1.0.3
# Cython  # for pycocotools https://github.com/cocodataset/cocoapi/issues/172
# pycocotools>=2.0  # COCO mAP
# roboflow
thop  # FLOPs computation

Writing requirements.txt


In [18]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install torch==1.10.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision==0.11.2+cu113 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [114]:
import glob
weight = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"

num = 5
img_dir = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/data"
img = glob.glob(f"{img_dir}/*")[num]
img

img = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/runs/detect/exp/APAC_fko0078.jpg"

In [119]:
!python detect.py  --weights $weight  --source $img

detect: weights=['/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt'], source=/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/runs/detect/exp/APAC_fko0078.jpg, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2022-12-26 torch 1.10.1+cu113 CPU

Fusing layers... 
Model Summary: 213 layers, 7034398 parameters, 0 gradients, 15.8 GFLOPs
tensor_shape: torch.Size([3, 480, 640])
tensor([[[0.16078, 0.13725, 0.11765,  ..., 0.12941, 0.12941, 0.12941],
         [0.15686, 0.13333, 0.11765,  ..., 0.12549, 0.12549, 0

# 必要な部分だけ抜粋

In [118]:
from models.common import DetectMultiBackend
#from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
#from utils.torch_utils import select_device, time_sync
from PIL import Image
import torch
from torchvision import models, transforms
import cv2
import numpy as np


device = 'cpu'
#device = select_device(device)
model = DetectMultiBackend(weight, device="cpu", dnn=False)
#stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
#imgsz = check_img_size([640], s=stride)  # check image size

class_names = {0:"infection", 1:"normal", 2:"non-infection", 3:"scar", 4:"tumor", 5:"deposit", 6:"APAC", 7:"lens opacity", 8:"bullous"}

transform = transforms.Compose([
            transforms.Resize(size=(480,640)),
            transforms.ToTensor(),
            # transforms.Normalize(
            #     mean=[0.5, 0.5, 0.5],
            #     std=[0.5, 0.5, 0.5]
            #    )
            ])

img_cv2 = cv2.imread(img)
img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
img_cv2 = np.ascontiguousarray(img_cv2)
img_tensor = torch.from_numpy(img_cv2).float()


#img_tensor = transform(img_np)
img_tensor /= 255
print(img_tensor.shape)
img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


pred = model(img_tensor, visualize=False, augment=False)

pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)

print(f"pred: {pred}")

Fusing layers... 
Model Summary: 213 layers, 7034398 parameters, 0 gradients, 15.8 GFLOPs


torch.Size([3, 480, 640])
pred: [tensor([[101.43054,  69.62360, 429.69128, 374.98340,   0.97141,   6.00000]])]


In [101]:
# probability
prob = pred[0][0][4].item()

# class
class_name = class_names[pred[0][0][5].item()]

print("診断は %s、確率は%.1f％です。" %(class_name, prob*100))

診断は normal、確率は86.3％です。


#**Export coreML including non-max suppression**

In [ ]:
# clone Yolov5 repo
import os
%cd /content
!git clone https://github.com/hietalajulius/yolov5.git
%cd yolov5
!pip install -r requirements.txt -r requirements-export.txt

In [56]:
weight_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"


In [ ]:
!python export-nms.py --include coreml --weights $weight_path


# **Deploy Streamlit**

In [ ]:
"""
必要ファイル
- yolo5_forstreamlit --> もともとのrepositoryから不要なファイルを削除してGitHubに上げる（※おそらく未発表のためprivateにする）
- app_streamlit.py
- model_streamlit.py
- requirement.txt --> もともとのものをベースに。適当に減らしてみて試行錯誤する

"""

In [42]:
%cd /content
!mkdir for_streamlit
%cd for_streamlit

/content
mkdir: cannot create directory ‘for_streamlit’: File exists
/content/for_streamlit


In [43]:
%%writefile requrements.txt


# pip install -r requirements.txt

# Base ----------------------------------------
matplotlib>=3.2.2
numpy>=1.18.5
#opencv-python-headless>=4.1.2
opencv-python-headless==4.6.0.66
Pillow>=7.1.2
PyYAML>=5.3.1
##requests>=2.23.0
scipy>=1.4.1
torch>=1.10.1
torchvision>=0.11.2
#tqdm>=4.41.0
tqdm>=4.63.2


# Logging -------------------------------------
tensorboard>=2.4.1
# wandb

# Plotting ------------------------------------
##pandas>=1.1.4
#seaborn>=0.11.0
seaborn>=0.12.1

# Export --------------------------------------
# coremltools>=4.1  # CoreML export
# onnx>=1.9.0  # ONNX export
# onnx-simplifier>=0.3.6  # ONNX simplifier
# scikit-learn==0.19.2  # CoreML quantization
# tensorflow>=2.4.1  # TFLite export
# tensorflowjs>=3.9.0  # TF.js export

# Extras --------------------------------------
# albumentations>=1.0.3
# Cython  # for pycocotools https://github.com/cocodataset/cocoapi/issues/172
# pycocotools>=2.0  # COCO mAP
# roboflow
thop  # FLOPs computation


Overwriting requrements.txt


In [44]:
%%writefile model_streamlit.py
from models.common import DetectMultiBackend
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
#from utils.torch_utils import select_device, time_sync
from PIL import Image
import torch
from torchvision import models, transforms

def predict(img):   #input: PIL image
    weight = "weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'
    model = DetectMultiBackend(weight, device="cpu", dnn=False)
    class_names = {0:"infection", 1:"normal", 2:"non-infection", 3:"scar", 4:"tumor", 5:"deposit", 6:"APAC", 7:"lens opacity", 8:"bullous"}
    transform = transforms.Compose([
                transforms.Resize(size=(640,640)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.5, 0.5, 0.5],
                    std=[0.5, 0.5, 0.5]
                    )
                ])

    img_tensor = transform(img)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応

    pred = model(img_tensor)
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)
    
    # probability
    prob = pred[0][0][4].item()

    # class
    class_name = class_names[pred[0][0][5].item()]

    return [class_name, prob]

Overwriting model_streamlit.py


In [45]:
%%writefile app_streamlit.py

import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from model_streamlit import predict

st.set_option("deprecation.showfileUploaderEncoding", False)

st.sidebar.title("CorneAI_web")
st.sidebar.write("")

img_source = st.sidebar.radio("画像のソースを選択してください。",
                              ("画像をアップロード", "カメラで撮影"))
if img_source == "画像をアップロード":
    img_file = st.sidebar.file_uploader("画像を選択してください。", type=["png", "jpg"])
elif img_source == "カメラで撮影":
    img_file = st.camera_input("カメラで撮影")

if img_file is not None:
    with st.spinner("推定中..."):
        img = Image.open(img_file).convert('RGB')
        st.image(img, caption="対象の画像", width=480)
        st.write("")

        # 予測
        results = predict(img)

        # 結果の表示
        st.subheader("判定結果")
        st.write("診断は %s、確率は%.1f％です。" %(result[0], result[1]*100))

Overwriting app_streamlit.py


In [46]:
#Test run

from google.colab import files
files.view("//gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forstreamlit")
files.view("app_streamlit.py")
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/bin/bash: streamlit: command not found
npx: installed 22 in 3.173s
your url is: https://social-ducks-walk-34-125-77-206.loca.lt
^C


In [58]:
# Interference
%cd /gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forstreamlit

def predict(img):   #input: PIL image
    weight = "weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'
    model = DetectMultiBackend(weight, device="cpu", dnn=False)
    class_names = {0:"infection", 1:"normal", 2:"non-infection", 3:"scar", 4:"tumor", 5:"deposit", 6:"APAC", 7:"lens opacity", 8:"bullous"}
    transform = transforms.Compose([
                transforms.Resize(size=(640,640)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.5, 0.5, 0.5],
                    std=[0.5, 0.5, 0.5]
                    )
                ])

    img_tensor = transform(img)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応

    pred = model(img_tensor)
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)
    
    # probability
    prob = pred[0][0][4].item()

    # class
    class_name = class_names[pred[0][0][5].item()]

    return [class_name, prob]

/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forstreamlit


In [63]:
predict(Image.open("/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/data/infection_amoeba_ehm0502_01.jpg"))

Fusing layers... 
Model Summary: 213 layers, 7034398 parameters, 0 gradients, 15.8 GFLOPs


['deposit', 0.8515667915344238]